In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", RuntimeWarning)

import sys
sys.path.append('../../dstorch/')

import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import os
import pandas as pd
from dstorch import transforms
from dstorch.train import train
import torch.backends.cudnn as cudnn

In [4]:
ids = list([x.split('.')[0] for x in list(next(os.walk("../output_data/with_scale_05/images/"))[2])])
classes = pd.read_csv("../data/classes.csv")
val_ids = list(classes.loc[classes['type'] == 'val', 'id'])
train_ids = list(set(ids) - set(val_ids))

In [5]:
train_transforms = transforms.DualCompose(
    [
        transforms.RandomCrop((256, 256)),
        transforms.VerticalFlip(),
        transforms.HorizontalFlip(),
        transforms.Transpose(),
        transforms.ShiftScaleRotate(scale_limit=0.2, shift_limit=0.05),
        transforms.Distort1(0.4, 0.4),
        transforms.ImageOnly(transforms.RandomBrightness(0.5)),
        transforms.ImageOnly(transforms.RandomContrast(0.5)),
        transforms.ImageOnly(transforms.RandomSaturation(0.5)),
        transforms.ImageOnly(transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ))
    ]
)

val_transforms = transforms.DualCompose(
    [
        transforms.ImageOnly(transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ))
    ]
)

In [6]:
cudnn.benchmark = True

In [7]:
train_args = {
    'model_name': 'TernausNet34_with_scale_05_vgg11',
    'network': 'UNet11',
    'nb_epoch': 600,
    'loss': 'BCEDiceLossMulti',
    'lr_args': {
        'adjust_init_lr': 0.0005,
        'adjust_lr_decay_factor': 0.6,
        'cycle_epochs_decay': 2,
        'cycle_init_lr': 0.001,
        'cycle_lr_decay_factor': 0.3,
        'global_cycle_lr_decay_factor': 0.9,
        'global_num_epochs_per_cycle': 100,
        'num_epochs_per_cycle': 5,
        'num_epochs_per_decay': 100
    }, 
    'model_dir': 'models', 
    'log_dir': 'logs', 
    'network_args': {
        'num_classes': 3, 
        'num_filters': 32, 
        'pretrained': True
    }, 
    'loss_args': {
        'weights': [0.6, 0.3, 0.1],
        'num_classes': 3
    },       
    'optimizer': 'Adam'
}

train(
    train_args, train_ids + val_ids, val_ids[:1], 
    "../output_data/with_scale_05/scaled_images/{}.png", "../output_data/with_scale_05/scaled_masks/{}.png", 
    16, 1, train_transforms, val_transforms, period=64
)

Epoch 0, lr 0.0005: 100% 43/43 [00:18<00:00,  2.28it/s, 0 cls=0.95551, 1 cls=1.24614, 2 cls=1.47115, bce=0.34439, dice=0.48975, loss=1.09426]


loss: 0.30534 | bce: 0.10567 | dice: 0.75094 | 0 cls: 0.18474 | 1 cls: 0.40056 | 2 cls: 0.68361


Epoch 1, lr 0.0005: 100% 43/43 [00:14<00:00,  2.90it/s, 0 cls=0.36840, 1 cls=0.49797, 2 cls=0.73977, bce=0.15343, dice=0.70495, loss=0.44441]


loss: 0.27145 | bce: 0.07014 | dice: 0.78031 | 0 cls: 0.24135 | 1 cls: 0.23194 | 2 cls: 0.50986


Epoch 2, lr 0.0005: 100% 43/43 [00:14<00:00,  2.90it/s, 0 cls=0.30447, 1 cls=0.40773, 2 cls=0.56246, bce=0.12079, dice=0.75719, loss=0.36125]


...

loss: 0.15152 | bce: 0.03744 | dice: 0.86763 | 0 cls: 0.11811 | 1 cls: 0.16677 | 2 cls: 0.29316


Epoch 598, lr 0.000177147: 100% 43/43 [00:15<00:00,  2.76it/s, 0 cls=0.15268, 1 cls=0.22277, 2 cls=0.31398, bce=0.06865, dice=0.86426, loss=0.18984]


loss: 0.14842 | bce: 0.03936 | dice: 0.87174 | 0 cls: 0.11368 | 1 cls: 0.16612 | 2 cls: 0.29028


Epoch 599, lr 5.3144100000000005e-05: 100% 43/43 [00:15<00:00,  2.82it/s, 0 cls=0.14273, 1 cls=0.21448, 2 cls=0.30401, bce=0.06606, dice=0.86987, loss=0.18038]


loss: 0.14923 | bce: 0.03929 | dice: 0.87118 | 0 cls: 0.11434 | 1 cls: 0.16654 | 2 cls: 0.29205
Finished: best loss 0.14619 on epoch 254
